In [85]:
from typing import Annotated
from langgraph.graph import StateGraph,START,END
from typing_extensions import TypedDict
from operator import add
from langgraph.checkpoint.memory import MemorySaver

# {"foo":[1]}


class State(TypedDict):
    # 自定义的归约
    # foo: Annotated[list[int],lambda x,y:x+y]
    # 做归约，改为追加而不是覆盖
    foo: Annotated[list[int],add]
    


def agent_01(state,config):

    new_state = {"foo":[state["foo"][-1]+1]}
    return new_state

_buider = StateGraph(State)

_buider.add_node("node_01", agent_01)
_buider.add_edge(START,"node_01")
_buider.add_edge("node_01",END)

# 加入记忆,必须要同一个进程
_memory=MemorySaver()
_graph = _buider.compile(checkpointer=_memory)

##### 设置配置

In [88]:
# 线程、递归限制（2以上）
_configurable={"thread_id":"2","recursion_limit":2,"configurable":{"session_id":"1"}}
_graph.invoke({"foo":[5]},config=_configurable)

{'foo': [5, 6, 5, 6]}

In [90]:
# _configurable={"thread_id":"2","recursion_limit":2,"configurable":{"session_id":"2"}}
_graph.invoke({"foo":[8]},config=_configurable)

{'foo': [5, 6, 5, 6, 6, 6, 8, 6]}